<a href="https://colab.research.google.com/github/aswa09/EVA-4/blob/master/EVA4_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,8, 3, padding=1),
            nn.BatchNorm2d(8),
            nn.Dropout(0.1)
        ) 

        self.conv2 = nn.Sequential(
            nn.Conv2d(8,16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1)
        )

        self.pool1=nn.MaxPool2d(2,2)

        self.conv3 = nn.Sequential(
            nn.Conv2d(16,16, 3),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(16,24, 3, padding=1),
            nn.BatchNorm2d(24),
            nn.Dropout(0.1)
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(24,24, 3),
            nn.BatchNorm2d(24),
            nn.Dropout(0.1)
        )

        self.pool2=nn.MaxPool2d(2,2)
        
        self.conv6 = nn.Sequential(
            nn.Conv2d(24,16, 3, padding=1),
            nn.BatchNorm2d(16)
        )

        self.conv7 = nn.Conv2d(16,10,3)

        self.avgp2d=nn.AdaptiveAvgPool2d((1,1))

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        #print(x.shape)
        x = self.pool2(F.relu(self.conv5(F.relu(self.conv4(F.relu(self.conv3(x)))))))
        #print(x.shape)
        x = F.relu(self.conv6(x))
        #print(x.shape)
        x = self.conv7(x)

        x=self.avgp2d(x)
        #print(x.shape)

        x = x.view(-1, 10)
        #print(x.shape)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
           Dropout-3            [-1, 8, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           1,168
       BatchNorm2d-5           [-1, 16, 28, 28]              32
           Dropout-6           [-1, 16, 28, 28]               0
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 16, 12, 12]           2,320
       BatchNorm2d-9           [-1, 16, 12, 12]              32
          Dropout-10           [-1, 16, 12, 12]               0
           Conv2d-11           [-1, 24, 12, 12]           3,480
      BatchNorm2d-12           [-1, 24, 12, 12]              48
          Dropout-13           [-1, 24, 12, 12]               0
           Conv2d-14           [-1, 24,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(19):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0665, Accuracy: 9803/10000 (98.03%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0382, Accuracy: 9897/10000 (98.97%)



  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0192, Accuracy: 9935/10000 (99.35%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0240, Accuracy: 9920/10000 (99.20%)



loss=0.006937086582183838 batch_id=937: 100%|██████████| 938/938 [00:21<00:00, 43.05it/s]



Test set: Average loss: 0.0197, Accuracy: 9941/10000 (99.41%)

